<a href="https://colab.research.google.com/github/cvonholly/technology_complexity/blob/main/add_patent_year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
!pip install python-epo-ops-client
import epo_ops
import xml.etree.ElementTree as ET

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 10.5 MB/s eta 0:00:00


In [3]:
# Mount Google Drive files. No need to rerun after running the first time

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [21]:
from google.colab import userdata
consumer_key = "5wpmow69fw4pjAXRrSKKVrlpHpuGgX9bFwlrVmX9XGAKKCHo"
consumer_secret = userdata.get('epo_secret')

In [4]:
df = pd.read_excel(r'/content/drive/My Drive/data/labeled_data/LeadAcid_random_1.xlsx')

In [5]:
df.head()

,appln_id,docdb_family_id,publn_nr,publn_auth,earliest_publn_date,appln_title,nb_citing_docdb_fam,tech,title,type,text,class,type.1,comment
0,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Abstract,The invention provided a new type of high capa...,product,apparatus,NaN
1,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 1,A high capacity colloidal storage battery whic...,product,apparatus,NaN
2,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 2,A high capacity colloidal storage battery acco...,product,composition,NaN
3,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 3,A high capacity colloidal storage battery acco...,product,apparatus,NaN
4,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Claim 4,A high capacity colloidal storage battery acco...,product,composition,NaN


In [6]:
pub_numbers = df[df['publn_auth']=='EP']['publn_nr'].unique()
df_unique = df[df['publn_auth']=='EP'].drop_duplicates(subset=['publn_nr'])
pub_numbers = pub_numbers.astype(str)
responses = {}  # dict to save responses

In [7]:
pub_numbers

array(['537373', '374894', '467861', '820112', '65348', '85109', '669670',
       '1447865', '2204866', '2819206', '2814088', '2811546', '2728650',
       '2897207', '2792534', '2124274'], dtype='<U21')

In [8]:
df_unique

,appln_id,docdb_family_id,publn_nr,publn_auth,earliest_publn_date,appln_title,nb_citing_docdb_fam,tech,title,type,text,class,type.1,comment
0,16880604,8207259,537373,EP,1993-04-21,"High capacity colloidal storage battery, a col...",4,Lead acid,"High capacity colloidal storage battery, a col...",Abstract,The invention provided a new type of high capa...,product,apparatus,NaN
18,323270,18150301,374894,EP,1990-06-27,Sealed tubular lead-acid battery.,12,Lead acid,Sealed tubular lead-acid battery,Abstract,A sealed tubular lead-acid battery is provided...,product,apparatus,NaN
22,16905684,11264833,467861,EP,1990-07-17,A lead-acid storage battery with tubular bipol...,6,Lead acid,A lead-acid storage battery with tubular bipol...,Abstract,A storage battery of the lead-acid type is dis...,product,apparatus,NaN
29,437680,16201150,820112,EP,1998-01-21,Enclosed lead storage battery,1,Lead acid,Enclosed lead storage battery,Abstract,Disclosed is an enclosed lead storage battery ...,product,apparatus,NaN
32,16500480,22996281,65348,EP,1982-11-24,BATTERY INTERCELL CONNECTOR MANIFOLD,10,Lead acid,Battery intercell connector manifold,Abstract,A multicell electric storage battery 2 having ...,product,apparatus,NaN
38,16511955,27523848,85109,EP,1983-02-17,LEAD STORAGE BATTERY AND METHOD OF PRODUCING T...,6,Lead acid,LEAD STORAGE BATTERY AND METHOD OF PRODUCING T...,Abstract,"Active substance layers (2, 4) are in close co...",product and process,NaN,NaN
41,17109421,11683633,669670,EP,1995-08-30,Agent for maintaining and recovering the funct...,8,Lead acid,Agent for maintaining and recovering the funct...,Abstract,The present invention relates to an agent for ...,product,composition,NaN
52,16127276,32681757,1447865,EP,2004-08-14,Improved single point watering apparatus for l...,12,Lead acid,Improved single point watering apparatus for l...,Abstract,The present invention is directed to a single ...,product,apparatus,NaN
63,274000000,41130147,2204866,EP,2010-06-24,Improved plant for the electrochemical formati...,5,Lead acid,Improved plant for the electrochemical formati...,Abstract,Improved plant for the electrochemical formati...,product,apparatus,NaN
78,408000000,48874764,2819206,EP,2014-12-31,Electrical accumulator,0,Lead acid,Electrical accumulator,Abstract,An electrical accumulator (1) comprising a box...,product,apparatus,NaN


In [10]:
def resp_to_pub_year(response):
  # Parse XML
  xml_response = response.text

  ns = {"epo": "http://www.epo.org/exchange"}  # Define the namespace
  root = ET.fromstring(xml_response)

  # Find the publication date using XPath
  pub_date = root.find(".//epo:publication-reference/epo:document-id/epo:date", ns)

  if pub_date is not None:
    publication_year = pub_date.text[:4]  # Extract only the year (YYYYMMDD format)
    print(f"Publication Year: {publication_year}")
    return publication_year
  else:
    print("Publication date not found.")
    return None

In [17]:
def get_EP_pub_date(pub_numbers):
  years = []
  client = epo_ops.Client(key=consumer_key, secret=consumer_secret)  # Instantiate client
  for p in pub_numbers:
    p = 'EP' + p
    try:
      response = client.published_data(  # Retrieve bibliography data
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Epodoc(p),
        # input = epo_ops.models.Docdb(p, 'EP', 'B1'),  # docdb, epodoc
        endpoint = 'biblio',  # optional, defaults to biblio in case of published_data
        constituents = []  # optional, list of constituents
      )
      responses[p] = response.text
      y = resp_to_pub_year(response)
      if y is not None:
        years.append(y)
      else:
        years.append(np.nan)
    except Exception as e:
      print('client call failed: ')
      print(e)
      years.append(np.nan)
      continue
  return years

In [18]:
years = get_EP_pub_date(pub_numbers)

client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
client call failed: 
404 Client Error: Not Found for url: https://ops.epo.org/3.2/rest-services/published-data/publication/epodoc/biblio
Publication Year: 2004
Publication Year: 

In [13]:
responses

{}

In [19]:
df_unique['api_years'] = years

In [20]:
df_unique[['publn_nr', 'earliest_publn_date', 'api_years']]

,publn_nr,earliest_publn_date,api_years
0,537373,1993-04-21,NaN
18,374894,1990-06-27,NaN
22,467861,1990-07-17,NaN
29,820112,1998-01-21,NaN
32,65348,1982-11-24,NaN
38,85109,1983-02-17,NaN
41,669670,1995-08-30,NaN
52,1447865,2004-08-14,2004
63,2204866,2010-06-24,2010
78,2819206,2014-12-31,2014


In [16]:
responses['2792534'] #.find('2014')

KeyError: '2792534'